<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Data Analytics 

### Interactive Shapley Value Demonstration in Python 


#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)


### Shapley Values

Here's a demonstration of Shapley values for subsurface modeling in Python. This is part of my Subsuface Machine Learning Course at the Cockrell School of Engineering at the University of Texas at Austin.  

#### Motivation

Complicated models are often required to model our natural settings, but have low interpretability. We have 2 choices to improve model interpretability: 

1. reduce the complexity of the models, but also reduce model accuracy

2. develop improved, agnostic (for any model) model diagnostics

Shapley values are the later, they treat the model as a black box and use a data subset, known as background, with predictor features $X_1,\ldots,X_m$ and model predictions $\hat{y}$ to discover structures between each predictor feature and the model response.

\begin{equation}
f(x_1,\ldots,x_m) = \sum_{i=1}^m \phi_i
\end{equation}

where $\phi_i$ is the Shapely value of the $i^{th}$ feature calculated by averaging over the combinatorial of marginal contributions for all feature orders and sequences.

* **order** - the number of previous features included before adding the $i^{th}$ feature

* **sequence** - the sequence of the addition of features


#### Description

Shapley values are adpated from game theory where the approach is used to calculate:

* allocating resources between ‘players’ based on a summarization of marginal contributions to the game. Dividing up winnings between all players. 

* contribution of each predictor feature to push the response prediction away from the mean value of the response over training

* an example of an marginal contribution include $f(x_2,x_1) - f(x_2)$ and $f(x_1) - E[y]$ to quantify the impact on the estimate of feature $X_1$, locally for an estimate with values $x_1$ and $x_2$.

Note, Shapley values are based completely on the model and do not access the accuracy of the model

* a poor model will potentially lead to incorrect interpretation with Shapley values

#### Variable Ranking

There are often many predictor features, input variables, available for us to work with for subsurface prediction. There are good reasons to be selective, throwing in every possible feature is not a good idea! In general, for the best prediction model, careful selection of the fewest features that provide the most amount of information is the best practice. 

Here's why:

* more variables result in more complicated workflows that require more professional time and have increased opportunity for blunders
* higher dimensional feature sets are more difficult to visualize
* more complicated models may be more difficult to interrogate, interpret and QC
* inclusion of highly redundant and colinear variables increases model instability and decreases prediction accuracy in testing
* more variables generally increase the computational time required to train the model and the model may be less compact and portable
* the risk of overfit increases with the more variables, more complexity

#### What is Feature Ranking?

Feature ranking is a set of metrics that assign relative importance or value to each feature with respect to information contained for inference and importance in predicting a response feature. There are a wide variety of possible methods to accomplish this. My recommendation is a **'wide-array'** approach with multiple metric, while understanding the assumptions and limitations of each metric.  

Here's the general types of metrics that we will consider for feature ranking.

1. Visual Inspection of Data Distributions and Scatter Plots
2. Statistical Summaries
3. Model-based
4. Recursive Feature Elimination 

Also, we should not neglect expert knowledge.  If additional information is known about physical processes, causation, reliability and availability of features this should be integrated into assigning feature ranks.

#### Import Packages

We will also need some standard packages. These should have been installed with Anaconda 3.

In [1]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
from sklearn.ensemble import RandomForestRegressor # random forest method
from ipywidgets import interactive        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

We have added Shapley value, let's install the shap package

In [2]:
#import sys                               # uncomment to install shapley if not installed
#!{sys.executable} -m pip install shap
import shap
shap.initjs()                             # enable javascript for Shapley plots

#### Set the Demonstration Parameters

Let's assume some parameters for our itneractive demonstrations.

In [3]:
n = 50; nb = 25                           # number of training and background data
b0 = 0.3

max_leaf_nodes = 5                        # default random forest parameters
num_tree = 300
max_features = 0.3

seed = 73073                              # random number seed for consistent interactive displays

xvals = np.linspace(-3,3,1000)            # grid sampling to visualize the models in 1D and 2D
x1values = np.linspace(-3, 3, 100)
x2values = np.linspace(-3, 3, 100)
x1grid, x2grid = np.meshgrid(x1values, x2values)
xgrid = np.stack((x1grid.flatten(), x2grid.flatten()), axis=1)

#### Experiment \#1: One Predictor Feature, Linear Relationship with Noise

Let's start very simple with one predictor feature and a linear relationship with noise added.

In [4]:
l = widgets.Text(value='                         One Predictor Feature Shapley Dependency Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

b1 = widgets.FloatSlider(min=0.00, max = 1.0, value = 0.5, step = 0.1, description = r'$b_1$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
b1.style.handle_color = 'red'
noise = widgets.FloatSlider(min=0.01, max = 1.0, value = 0.20, step = 0.01, description = r'$\sigma_{noise}$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
noise.style.handle_color = 'red'

ui1 = widgets.HBox([b1,noise],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(b1,noise):
    np.random.seed(seed=seed)
    features = ['X1']
    X1 = np.random.normal(loc = 0.0,scale = 1.0,size = n)
    X1b = np.random.normal(loc = 0.0,scale = 1.0,size = nb)
    y = b1*X1 + b0 + np.random.normal(loc = 0.0, scale = noise, size = n)
    yb = b1*X1b + b0 + np.random.normal(loc = 0.0, scale = noise, size = nb)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
    #axes[0].scatter(X1,y,color = 'red', edgecolor = 'black',label = 'training',alpha = 0.3)
    axes[0].scatter(X1b,yb,color = 'white', edgecolor = 'black',label = '$y_{background}$', alpha = 0.3)
    axes[0].set_xlim(-3,3); axes[0].set_ylim(-2,2)
    
    random_forest = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=seed,n_estimators=num_tree, max_features=max_features)
    random_forest.fit(X = X1.reshape(-1, 1), y = y.reshape(-1,))
    y_hat = random_forest.predict(xvals.reshape(-1, 1))
    yb_hat = random_forest.predict(X1b.reshape(-1, 1))
    axes[0].plot(xvals,y_hat,color='black', label = '$\\widehat{y}$', alpha = 0.3)
    axes[0].plot([-3,3],[np.average(y),np.average(y)],'--',color = 'red',alpha = 0.3)
    axes[0].scatter(X1b,yb_hat,color = 'blue', edgecolor = 'black',label = '$\\widehat{y}_{background}$')
    axes[0].legend(); axes[0].set_xlabel('$X_1$'); axes[0].set_ylabel('y')
    axes[0].annotate(xy = [-2.8,np.average(y)+0.2],text='$E[y]$'); axes[0].set_title('Data and Model')
    
    shap_values = shap.TreeExplainer(random_forest).shap_values(X1b.reshape(-1, 1))
    shap.dependence_plot(ax = axes[1],ind = features[0],show=False,feature_names = features, shap_values = shap_values, features = X1b.reshape(-1, 1), cmap = plt.cm.inferno)
    axes[1].plot(xvals,y_hat-np.average(y),color='black',alpha = 0.2,label = '$\\Delta \widehat{y}$')
    axes[1].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[1].legend(); axes[1].set_title('Dependency Plot')
    axes[1].set_xlim(-3,3); axes[1].set_ylim(-2,2)

interactive_plot = widgets.interactive_output(f_make, {'b1':b1,'noise':noise})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

<>:36: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
<>:36: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
/tmp/ipykernel_247073/44210121.py:36: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
  axes[1].plot(xvals,y_hat-np.average(y),color='black',alpha = 0.2,label = '$\\Delta \widehat{y}$')


Output()

We can make the following observations:

* the Shapley value is just the $f(X_1) - E[y]$ marginal contribution. 

* we can observe that the Shapley values are the departure of the predictions of $y$ form the global average of $\overline{y}$

* noise disrupts the original linear structure of the relationship

#### Experiment \#2: Two Predictor Features, Multilinear Relationship with Noise and Control Model Hyperparameter

Now let's add another feature $X_2$ and include control of the model hyperparameter, number of leaf nodes, so we can underfit the model.

In [5]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                         Two Predictor Features Shapley Dependency Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

b1 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_1$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
b1.style.handle_color = 'red'
b2 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_2$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
b2.style.handle_color = 'red'
noise = widgets.FloatSlider(min=0.01, max = 5.0, value = 0.20, step = 0.1, description = '$\\sigma_{noise}$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
noise.style.handle_color = 'red'
nodes = widgets.IntSlider(min=2, max = 100, value = 5, step = 1, description = 'leaf nodes',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
nodes.style.handle_color = 'red'

ui1 = widgets.HBox([b1,b2,noise,nodes],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})
num_tree = 5

def f_make(b1,b2,noise,nodes):
    np.random.seed(seed=seed)
    max_leaf_nodes = 25; n = 300; nb = 200
    features = ['X1','X2']
    X1 = np.random.normal(loc = 0.0,scale = 1.0,size = n)
    X1b = np.random.normal(loc = 0.0,scale = 1.0,size = nb)
    X2 = np.random.normal(loc = 0.0,scale = 1.0,size = n)
    X2b = np.random.normal(loc = 0.0,scale = 1.0,size = nb)
    y = b2*X2 + b1*X1 + b0 + np.random.normal(loc = 0.0, scale = noise, size = n)
    yb = b2*X2b + b1*X1b + b0 + np.random.normal(loc = 0.0, scale = noise, size = nb)
    
    X = np.stack((X1, X2), axis=1)
    Xb = np.stack((X1b, X2b), axis=1)
    
    fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(20, 5), sharey=False, gridspec_kw = {"wspace":0.3})
    
    random_forest = RandomForestRegressor(max_leaf_nodes=nodes, random_state=seed,n_estimators=num_tree, max_features=max_features)
    random_forest.fit(X = X, y = y.reshape(-1,))
    y_hat = random_forest.predict(xgrid)
    yb_hat = random_forest.predict(Xb)
    ymin = np.min(y_hat.reshape([100,100])); ymax = np.max(y_hat.reshape([100,100]))
    
    #axes[0].scatter(X1,X2,c = y,edgecolor = 'black',marker = 'v',s = 20, alpha = 0.3, cmap = plt.cm.inferno,label = 'training')
    axes[0].scatter(X1b,X2b,c = yb_hat,edgecolor = 'black',s = 20, alpha = 0.8, cmap = plt.cm.inferno,label = 'background')
    ct = axes[0].contour(x1grid, x2grid,y_hat.reshape([100,100]),levels = 20, cmap = plt.cm.inferno, alpha = 0.2, vmin = ymin, vmax = ymax)
    axes[0].legend(); axes[0].set_xlabel('$X_1$'); axes[0].set_ylabel('$X_2$')
    axes[0].set_title('Data and Model')
    axes[0].set_xlim(-3,3)
    fig.colorbar(ct, ax=axes[0])
    
    shap_values = shap.TreeExplainer(random_forest).shap_values(Xb)
    shap.dependence_plot(ax = axes[1],ind = features[0],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[1].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[1].set_title('Dependency Plot')
    #axes[1].set_xlim(-3,3); axes[1].set_ylim(-1*ymax,ymax)
    axes[1].set_xlim(-3,3); axes[1].set_ylim(-1.5,1.5)    

    shap.dependence_plot(ax = axes[2],ind = features[1],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[2].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[2].set_title('Dependency Plot')
    #axes[2].set_xlim(-3,3); axes[2].set_ylim(-1*ymax,ymax)
    axes[2].set_xlim(-3,3); axes[2].set_ylim(-1.5,1.5)
    
    axes[0].set_ylim(-3,3)

interactive_plot = widgets.interactive_output(f_make, {'b1':b1,'b2':b2,'noise':noise,'nodes':nodes})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

We can make the following observations:

* the Shapley value now includes:

    * $f(X_1) - E[y]$ and $f(X_1,X_2) - f(X_2)$ marginal contributions for the $X_1$ Shapley value
    
    * $f(X_2) - E[y]$ and $f(X_1,X_2) - f(X_1)$ marginal contributions for the $X_2$ Shapley value
   
* as a result the Shapley values extract the individual contribution structures for each feature even when there is significant conditional variance, $\sigma_{y | X_1}$ and $\sigma_{y | X_2}$ 

* the influence of the features over the range of departures from the global mean, $E{y}$, are approximately indepedendent (if model complexity is well tuned)

* noise still disrupts the original linear structure of the relationship 

* noise and complexity (large number of leaf nodes) can result in an overfit model, adding further noise to the Shapley values

#### Experiment \#3: Two Predictor Features, Non-linear Relationship with Noise and Control Model Hyperparameter

Now let's add a square term to the $X_1$ feature and retain control of the model hyperparameter, number of leaf nodes, so we can underfit the model.

In [6]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                         Two Predictor Features Shapley Dependency Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

b1 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_1$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
b1.style.handle_color = 'red'
b2 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_2$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
b2.style.handle_color = 'red'
noise = widgets.FloatSlider(min=0.01, max = 5.0, value = 0.20, step = 0.01, description = '$\\sigma_{noise}$',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
noise.style.handle_color = 'red'
nodes = widgets.IntSlider(min=2, max = 1000, value = 50, step = 1, description = 'leaf nodes',orientation='horizontal',layout=Layout(width='300px', height='50px'),continuous_update=False)
nodes.style.handle_color = 'red'

ui1 = widgets.HBox([b1,b2,noise,nodes],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(b1,b2,noise,nodes):
    np.random.seed(seed=seed)
    max_leaf_nodes = 25; n = 300; nb = 200
    features = ['X1','X2']
    X1 = np.random.normal(loc = 0.0,scale = 1.0,size = n)
    X1b = np.random.normal(loc = 0.0,scale = 1.0,size = nb)
    X2 = np.random.normal(loc = 0.0,scale = 1.0,size = n)
    X2b = np.random.normal(loc = 0.0,scale = 1.0,size = nb)
    y = b2*X2 + b1*X1*X1 + b0 + np.random.normal(loc = 0.0, scale = noise, size = n)
    yb = b2*X2b + b1*X1b*X1b + b0 + np.random.normal(loc = 0.0, scale = noise, size = nb)
    
    X = np.stack((X1, X2), axis=1)
    Xb = np.stack((X1b, X2b), axis=1)
    
    fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(20, 5), sharey=False, gridspec_kw = {"wspace":0.3})
    
    random_forest = RandomForestRegressor(max_leaf_nodes=nodes, random_state=seed,n_estimators=num_tree, max_features=max_features)
    random_forest.fit(X = X, y = y.reshape(-1,))
    y_hat = random_forest.predict(xgrid)
    yb_hat = random_forest.predict(Xb)
    ymin = np.min(y_hat.reshape([100,100])); ymax = np.max(y_hat.reshape([100,100]))
    
    #axes[0].scatter(X1,X2,c = y,edgecolor = 'black',marker = 'v',s = 20, alpha = 0.3, cmap = plt.cm.inferno,label = 'training')
    axes[0].scatter(X1b,X2b,c = yb,edgecolor = 'black',s = 20, alpha = 0.8, cmap = plt.cm.inferno,label = 'background')
    ct = axes[0].contour(x1grid, x2grid,y_hat.reshape([100,100]),levels = 20, cmap = plt.cm.inferno, alpha = 0.2, vmin = ymin, vmax = ymax)
    axes[0].legend(); axes[0].set_xlabel('$X_1$'); axes[0].set_ylabel('$X_2$')
    axes[0].set_title('Data and Model')
    axes[0].set_xlim(-3,3)
    fig.colorbar(ct, ax=axes[0])
    
    shap_values = shap.TreeExplainer(random_forest).shap_values(Xb)
    shap.dependence_plot(ax = axes[1],ind = features[0],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[1].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[1].set_title('Dependency Plot')
    axes[1].set_xlim(-3,3); axes[1].set_ylim(-1*ymax,ymax)
    
    shap.dependence_plot(ax = axes[2],ind = features[1],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[2].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[2].set_title('Dependency Plot')
    axes[2].set_xlim(-3,3); axes[2].set_ylim(-1*ymax,ymax)
    
interactive_plot = widgets.interactive_output(f_make, {'b1':b1,'b2':b2,'noise':noise,'nodes':nodes})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

We can make the following observations:

* the Shapley values are able to reveal the linear ($X_2$) and non-linear, quadratic ($X_1$) relationships:

    * $f(X_1) - E[y]$ and $f(X_1,X_2) - f(X_2)$ marginal contributions for the $X_1$ Shapley value
    
    * $f(X_2) - E[y]$ and $f(X_1,X_2) - f(X_1)$ marginal contributions for the $X_2$ Shapley value
   
* as a result the Shapley values extract the individual contribution structures for each feature even when there is significant conditional variance, $\sigma_{y | X_1}$ and $\sigma_{y | X_2}$ 

* the influence of the features over the range of departures from the global mean, $E{y}$, are approximately indepedendent (if model complexity is well tuned)

* noise still disrupts the original linear structure of the relationship 

* noise and complexity (large number of leaf nodes) can result in an overfit model, adding further noise to the Shapley values

#### Experiment \#4: Two Predictor Features, Multilinear Relationship with Correlation, Noise and Control Model Hyperparameter

Now let's add correlation between the predictor features, $X_1$ and $X_2$, and retain control of the model hyperparameter, number of leaf nodes, while adding number of trees so we can underfit the model.

In [7]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                         Two Predictor Features Shapley Dependency Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

b1 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_1$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
b1.style.handle_color = 'red'
b2 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_2$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
b2.style.handle_color = 'red'
corr = widgets.FloatSlider(min=-1.0, max = 1.0, value = 0.0, step = 0.1, description = '$\\rho$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
corr.style.handle_color = 'red'
noise = widgets.FloatSlider(min=0.01, max = 5.0, value = 0.20, step = 0.1, description = '$\\sigma_{noise}$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
noise.style.handle_color = 'red'
nodes = widgets.IntSlider(min=2, max = 100, value = 50, step = 1, description = 'leaf nodes',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
nodes.style.handle_color = 'red'
ntree = widgets.IntSlider(min=1, max = 100, value = 50, step = 1, description = '# tree',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
ntree.style.handle_color = 'red'

ui1 = widgets.HBox([b1,b2,corr],kwargs = {'justify_content':'center'}) 
ui2 = widgets.HBox([noise,nodes,ntree],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1,ui2],kwargs = {'justify_content':'center'})

def f_make(b1,b2,corr,noise,nodes,ntree):
    np.random.seed(seed=seed)
    max_leaf_nodes = 25; n = 300; nb = 200
    features = ['X1','X2']
    
    mean = [0.0,0.0]; stdev = [1.0,1.0]
    cov = np.array([1.0,corr,corr,1.0]); cov = cov.reshape((2,2))
    X = np.random.multivariate_normal(mean, cov, size = n)
    X1 = X[:,0]; X2 = X[:,1]
    Xb = np.random.multivariate_normal(mean, cov, size = nb)
    X1b = Xb[:,0]; X2b = Xb[:,1]

    y = b2*X2 + b1*X1 + b0 + np.random.normal(loc = 0.0, scale = noise, size = n)
    yb = b2*X2b + b1*X1b + b0 + np.random.normal(loc = 0.0, scale = noise, size = nb)
    
    fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(20, 5), sharey=False, gridspec_kw = {"wspace":0.3})
    
    random_forest = RandomForestRegressor(max_leaf_nodes=nodes, random_state=seed,n_estimators=ntree, max_features=max_features)
    random_forest.fit(X = X, y = y.reshape(-1,))
    y_hat = random_forest.predict(xgrid)
    yb_hat = random_forest.predict(Xb)
    #ymin = np.min(y_hat.reshape([100,100])); ymax = np.max(y_hat.reshape([100,100]))
    #ymin = np.min(y); ymax = np.max(y)
    
    axes[0].scatter(X1,X2,c = y,edgecolor = 'black',marker = 'v',s = 20, alpha = 0.3, cmap = plt.cm.inferno,label = 'training')
    axes[0].scatter(X1b,X2b,c = yb,edgecolor = 'black',s = 20, alpha = 0.8, cmap = plt.cm.inferno,label = 'background')
    #ct = axes[0].contour(x1grid, x2grid,y_hat.reshape([100,100]),levels = 20, cmap = plt.cm.inferno, alpha = 0.2, vmin = ymin, vmax = ymax)
    ct = axes[0].contour(x1grid, x2grid,y_hat.reshape([100,100]),levels = 20, cmap = plt.cm.inferno, alpha = 0.2, vmin = -3, vmax = 3)
    axes[0].legend(); axes[0].set_xlabel('$X_1$'); axes[0].set_ylabel('$X_2$')
    axes[0].set_title('Data and Model')
    axes[0].set_xlim(-3,3)
    fig.colorbar(ct, ax=axes[0])
    
    shap_values = shap.TreeExplainer(random_forest).shap_values(Xb)
    shap.dependence_plot(ax = axes[1],ind = features[0],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[1].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[1].set_title('Dependency Plot')
    #axes[1].set_xlim(-3,3); axes[1].set_ylim(ymin,ymax)
    axes[1].set_xlim(-3,3); axes[1].set_ylim(-3,3)
    
    shap.dependence_plot(ax = axes[2],ind = features[1],show=False,feature_names = features, shap_values = shap_values, features = Xb, cmap = plt.cm.inferno)
    axes[2].plot([-3,3],[0,0],'--',color = 'red', alpha = 0.3)
    axes[2].set_title('Dependency Plot')
    #axes[2].set_xlim(-3,3); axes[2].set_ylim(ymin,ymax)
    axes[2].set_xlim(-3,3); axes[2].set_ylim(-3,3)
    
    axes[0].set_ylim(-3,3)

interactive_plot = widgets.interactive_output(f_make, {'b1':b1,'b2':b2,'corr':corr,'noise':noise,'nodes':nodes,'ntree':ntree})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

We can add the following observations:

* correlation between the predictor features, $X_1$ and $X_2$, adds structure and ordering to the dependency plots

* this is evident with ordering and stratification relationships 
      
#### Experiment \#5: Global Shapley for Two Predictor Features, Multilinear Relationship with Correlation, Noise and Control Model Hyperparameter

Now let's take the previous experiment and apply it to global Shapley measures.

In [8]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                         Global Shapley Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

b1 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_1$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
b1.style.handle_color = 'red'
b2 = widgets.FloatSlider(min=0.00, max = 5.0, value = 0.5, step = 0.1, description = '$b_2$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
b2.style.handle_color = 'red'
corr = widgets.FloatSlider(min=-1.0, max = 1.0, value = 0.0, step = 0.1, description = '$\\rho$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
corr.style.handle_color = 'red'
noise = widgets.FloatSlider(min=0.01, max = 5.0, value = 0.20, step = 0.1, description = '$\\sigma_{noise}$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
noise.style.handle_color = 'red'
nodes = widgets.IntSlider(min=2, max = 100, value = 50, step = 1, description = 'leaf nodes',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
nodes.style.handle_color = 'red'
ntree = widgets.IntSlider(min=1, max = 100, value = 50, step = 1, description = '# tree',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
ntree.style.handle_color = 'red'

ui1 = widgets.HBox([b1,b2,corr],kwargs = {'justify_content':'center'}) 
ui2 = widgets.HBox([noise,nodes,ntree],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1,ui2],kwargs = {'justify_content':'center'})

def f_make(b1,b2,corr,noise,nodes,ntree):
    np.random.seed(seed=seed)
    max_leaf_nodes = 25; n = 300; nb = 200
    features = ['X1','X2']
    
    mean = [0.0,0.0]; stdev = [1.0,1.0]
    cov = np.array([1.0,corr,corr,1.0]); cov = cov.reshape((2,2))
    X = np.random.multivariate_normal(mean, cov, size = n)
    X1 = X[:,0]; X2 = X[:,1]
    Xb = np.random.multivariate_normal(mean, cov, size = nb)
    X1b = Xb[:,0]; X2b = Xb[:,1]

    y = b2*X2 + b1*X1 + b0 + np.random.normal(loc = 0.0, scale = noise, size = n)
    yb = b2*X2b + b1*X1b + b0 + np.random.normal(loc = 0.0, scale = noise, size = nb)
    
    fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize=(20, 5), sharey=False, gridspec_kw = {"wspace":0.3})
    
    random_forest = RandomForestRegressor(max_leaf_nodes=nodes, random_state=seed,n_estimators=ntree, max_features=max_features)
    random_forest.fit(X = X, y = y.reshape(-1,))
    y_hat = random_forest.predict(xgrid)
    yb_hat = random_forest.predict(Xb)
    #ymin = np.min(y_hat.reshape([100,100])); ymax = np.max(y_hat.reshape([100,100]))
    ymin = np.min(y); ymax = np.max(y)
    
    shap_values = shap.TreeExplainer(random_forest).shap_values(Xb)

    plt.subplot(131)
    shap.summary_plot(show=False,feature_names = features, shap_values = shap_values, features = Xb, plot_type="bar",color = "gray", cmap = plt.cm.inferno)
    plt.ylabel('Predictor Features')
    
    plt.subplot(132)
    shap.summary_plot(show=False,feature_names = features, shap_values = shap_values, features = Xb,cmap = plt.cm.inferno)
    
    plt.subplot(133)
    shap.summary_plot(show=False,feature_names = features, shap_values = shap_values, features = Xb,plot_type = "violin")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.2, top=1.2, wspace=0.2, hspace=0.2)
    plt.show()

interactive_plot = widgets.interactive_output(f_make, {'b1':b1,'b2':b2,'corr':corr,'noise':noise,'nodes':nodes,'ntree':ntree})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot


Output()

We can observed the competition for global feature importance, and the impact of model overfit.

#### Comments

This was a basic interactive demonstration of Shapley values. 

I have other demonstrations on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations, trend modeling and many other workflows available at [Python Demos](https://github.com/GeostatsGuy/PythonNumericalDemos) and a Python package for data analytics and geostatistics at [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy). 
  
I hope this was helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)
